In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define our example directories and files
IMAGE_SIZE = [150, 150]

train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   validation_split=0.15,
                                   featurewise_center=False,
                                   samplewise_center=False,
                                   featurewise_std_normalization=False,
                                   samplewise_std_normalization=False,
                                   zca_whitening=False,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   vertical_flip=False
                                  )

# Note that the testing data should not be augmented!
test_datagen = ImageDataGenerator(rescale = 1.0/255.0)

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory('/kaggle/input/dataset-for-different-eye-disease/DATASET_101/Train',
                                                    target_size = (150, 150),
                                                    batch_size = 64,
                                                    subset = "training",
                                                    class_mode = 'categorical',
                                                    #color_mode = 'grayscale'       # Comment
                                                   )

# Flow validation images in batches of 32 using train_datagen generator
validation_generator = train_datagen.flow_from_directory('/kaggle/input/dataset-for-different-eye-disease/DATASET_101/Train',
                                                         target_size = (150, 150),
                                                         batch_size = 64,
                                                         subset = "validation",
                                                         class_mode = 'categorical',
                                                         #color_mode = 'grayscale'       # Comment
                                                        )

# Flow testing images in batches of 32 using test_datagen generator
test_generator = test_datagen.flow_from_directory('/kaggle/input/dataset-for-different-eye-disease/DATASET_101/Test',
                                                  target_size = (150, 150),
                                                  batch_size  = 64,
                                                  class_mode  = 'categorical',
                                                  #color_mode = 'grayscale',       # Comment
                                                  shuffle = False
                                                 )

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (7,7), activation='relu', input_shape=(150, 150, 3), padding="same", name="L1"),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (7,7), activation='relu', name="L2"),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (7,7), activation='relu', name="L3"),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    #tf.keras.layers.Conv2D(128, (7,7), activation='relu', name="L4"),
    #tf.keras.layers.MaxPooling2D(2,2),

    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),

    # 256 neuron hidden layer
    tf.keras.layers.Dense(256, activation='relu', name="dense1" ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),
    # 128 neuron hidden layer
    tf.keras.layers.Dense(128, activation='relu', name="dense2" ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),

    # 6 neuron output layer
    tf.keras.layers.Dense(6, activation='softmax')
])

# Print the model summary
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import time

start = time.time()

# Compiling the CNN  # Set the training parameters
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
              metrics = ['accuracy', #keras.metrics.Precision(), keras.metrics.Recall(),
                         #keras.metrics.SpecificityAtSensitivity(0.5),
                         #keras.metrics.SensitivityAtSpecificity(0.5)
                         ])

# Training the CNN on the Training set and evaluating it on the Test set
# Stop training if loss doesn't keep decreasing.
model_es = EarlyStopping(monitor = 'val_loss', min_delta = 1e-11, patience = 8, verbose = 1)
model_rlr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 3, verbose = 1)

# Automatically saves the best weights of the model, based on best val_accuracy
model_mcp = ModelCheckpoint(filepath = 'model_1_CNN.keras', monitor = 'val_accuracy',
                             save_best_only = True, verbose = 1)

history = model.fit(train_generator, validation_data = validation_generator,
                    epochs = 50, callbacks = [model_es, model_rlr, model_mcp]       # CHANGE number of epochs
                    #steps_per_epoch=62, validation_steps=6
                    )

end = time.time()
elapsed = end - start
print("Total Time:", elapsed)

In [ ]:
import matplotlib.pyplot as plt

# accuracies
plt.figure(figsize=(6,4))
plt.plot(history.history['accuracy'], 'r', label='Training accuracy')
plt.plot(history.history['val_accuracy'], 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('No. of Iteration (Epochs)')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.savefig('train_val_acc.png')

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Generate predictions using the model
Y_pred = model.predict(test_generator)  # Assuming test_generator yields batches of data
y_pred = np.argmax(Y_pred, axis=1)

# Print confusion matrix
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

# Print classification report
target_names = ['ACRIMA', 'Glaucoma', 'ODIR-5K', 'ORIGA', 'cataract', 'retina_disease']
print('Classification Report')
print(classification_report(test_generator.classes, y_pred, target_names=target_names, digits=4))

In [ ]:
import math
number_of_examples = len(test_generator.filenames)
number_of_generator_calls = math.ceil(number_of_examples / (1.0 * 64))
# 1.0 above is to skip integer division

test_labels = []
test_images = []

for i in range(0,int(number_of_generator_calls)):
    test_labels.extend(np.array(test_generator[i][1]))

for i in range(0,int(number_of_generator_calls)):
    test_images.extend(np.array(test_generator[i][0]))
    %matplotlib inline
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
predictions = model.predict(test_generator)
new_labels = []
for i in range(0,1395):
  new_labels.append(np.argmax(predictions[i]))
newtest_labels = []
for i in range(0,1395):
  newtest_labels.append(np.argmax(test_labels[i]))
cm = confusion_matrix(y_true=newtest_labels, y_pred=new_labels)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, auc

# Calculate accuracy score
score = metrics.accuracy_score(newtest_labels, new_labels)
print("Accuracy score: {}".format(score))

# Generate prediction probabilities
pred_prob = model.predict(test_generator)

# Calculate ROC AUC score
c = roc_auc_score(newtest_labels, pred_prob, multi_class='ovo')
print("AUC:", c)

# Compute ROC curve and ROC area for each class
fpr = {}
tpr = {}
roc_auc = {}
lw = 2
precision = {}
recall = {}

for i in range(6):
    fpr[i], tpr[i], _ = roc_curve(newtest_labels, pred_prob[:, i], pos_label=i)
    precision[i], recall[i], _ = roc_curve(newtest_labels, pred_prob[:, i], pos_label=i)
    roc_auc[i] = auc(fpr[i], tpr[i])
    plt.plot(recall[i], precision[i], lw=2, label='class {}'.format(i))

plt.xlabel("recall")
plt.ylabel("precision")
plt.legend(loc="best")
plt.title("precision vs. recall curve")
plt.savefig('prc.png')
plt.show()

n_classes = 6

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at these points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally, average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()

colors = cycle(['green', 'darkorange', 'cornflowerblue', 'yellow' ])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw, label='ROC curve of class {0} (area = {1:0.4f})'
                                                       ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.savefig('ssr.png')
plt.show()